<a href="https://colab.research.google.com/github/Seunghoon-Yi/20-spring-ML/blob/master/AS4_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M2608.001300 기계학습 기초 및 전기정보 응용<br> Assignment 4: Hidden Markov Model (HMM)

In [ ]:
# Code from Chapter 16 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2008, 2014

# A basic Hidden Markov Model

## Setup
Check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead).

In [ ]:
# Python >=3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
from pandas import DataFrame

# to make this notebook's output stable across runs
np.random.seed(4)

## Problem 1. Evaluation problem
Get the probability of seeing an observation. <br>
We can use the following 2 algorithms:
<br>
* <font color=blue>*Forward algorithm*</font> (dynamic programming)
* <font color=blue>*Backward algorithm*</font> (dynamic programming)

In [ ]:
scaling = False

In [ ]:
"""
Let's denote the parameters of HMM model as,

pi : initial state probability
a : transition probability
b : emission probability
obs : observation sequence
"""
def HMMfwd(pi, a, b, obs):

	# TODO: Forward algorithm
	####################### YOUR CODE HERE #######################
	N_st = len(a)
	N_t = len(obs)
####alpha initialization, kibun이 observation in each time t with state s_t. Q) 그럼 얘는 뭘 하고 있었는가(b)?
# 그러니까 우리가 구해야 하는건 뭘 하고 있었는지의 ML estimation sequence
	alpha = np.zeros((N_st, N_t))
	alpha[:,0] = pi*b[obs[0],:]

	c = np.ones(N_t)                                        #check dimension in #3
	for T in range(1, N_t):									#time dimension
		for S in np.arange(N_st):
			alpha[S,T] = b[obs[T],S]*np.sum(alpha[:,T-1]*a[S,:]) #모든 state에서 특정 state로 전이확률
		if scaling:
			c[T] = np.sum(alpha[:,T])
			alpha[:,T]/=c[T]					#columnwise normalization  
	##############################################################
	return alpha,c

def HMMbwd(a, b, obs, c):

	# TODO: Backward algorithm
	####################### YOUR CODE HERE #######################
	N_st = np.shape(b)[0]
	N_t = len(obs)
 
	beta = np.zeros((N_st, N_t))
	beta[:,N_t-1] = 1									  #final column = 1
	for T in range(N_t-2, -1, -1):
		for S in range(N_st):
			beta[S,T] = np.sum(beta[:,T+1]*b[:,obs[T+1]]*a[S,:]) #한 state에서 모든 state로 전이확률
	for T in range(N_t):
		beta[:,T]/=c[T]
	##############################################################
	return beta

## Problem 2. Decoding problem
Get the underlying state sequence of an observation. <br>
We can use the <font color=blue>*Viterbi algorithm*</font> (dynamic programming).

In [ ]:
def Viterbi(pi, a, b, obs):
	# TODO: Viterbi algorithm
	####################### YOUR CODE HERE #######################
	N_st = len(a)
	N_t = len(obs)
	
	path = np.zeros(N_t)
	delta = np.zeros((N_st,N_t))							#states*timestep
	psi = np.zeros((N_st,N_t))
##initialization
	delta[:,0] = pi*b[obs[0],:]
	psi[:,0] = 0
	for t in range(1,N_t):
		for s_n in range(N_st):
			delta[s_n,t] = np.max(delta[:,t-1]*a[s_n,:])*b[obs[t],s_n]
			psi[s_n,t] = np.argmax(delta[:,t-1]*a[s_n,:]) 	  #maximum connection value index
	
	path[N_t-1] = np.argmax(delta[:,N_t-1])                   #maximum probable 'final' path의 '종착점' index
	for t in range(N_t-2, -1, -1):							  #path 맨마지막 앞부터
		#print(path[t+1])
		path[t] = psi[int(path[t+1]), t+1]				
	##############################################################
	return path, delta, psi

## Problem 3. Learning problem
Get the best parameters for the model. <br>
We can use the following 3 algorithms:
* *MLE* (maximum likelihood estimation)
* *Viterbi training* (different from Viterbi decoding)
* *Baum Welch algorithm* (a.k.a. forward-backward algorithm, uses expectation-maximization)

Here we implement the <font color=blue>*Baum Welch algorithm*</font>.

In [ ]:
def BaumWelch(obs,nStates):

	T = np.shape(obs)[0]
	xi = np.zeros((nStates,nStates,T)) 

		########
	  # Time # #
	######## # #
	######## #
	########  

	# depth = T이므로 t는 항상 모든 list idx 뒷자리에!
    
	# Initialise pi, a, b randomly
	pi = 1./nStates*np.ones((nStates))
	a = np.random.rand(nStates,nStates)
	b = np.random.rand(nStates,np.max(obs)+1)         

	tol = 1e-5
	error = tol+1
	iter_max = 300
	iter = 0

	# TODO: Baum Welch algorithm
	####################### YOUR CODE HERE #######################
	while ((error>tol) and (iter<iter_max)):
		iter+=1
		pi_old, a_old, b_old = pi.copy(),a.copy(),b.copy()
					 ######## Estimation ########
		alpha, c = HMMfwd(pi,a,b,obs)
		beta = HMMbwd(a, b, obs, c)
		for t in range(T):
			for si in range(nStates):
				for sj in range(nStates):
					if t == (T-1):                        
						xi[si,sj,t] = alpha[si,t]*a[si,sj]
					else: 											#as we initialize the last beta as 1, with no b
						xi[si,sj,t] = alpha[si,t]*a[si,sj]*b[sj,obs[t+1]]*beta[sj,t+1] #ppt 87						
			xi[:,:,t]/=np.sum(xi[:,:,t])
   				    ######## Maximization ########
		for si in range(nStates):
			#pi[si] = gamma(t=1(init), si = si)
			#       = xi sum in sj
			pi[si] = np.sum(xi[si,:,0])
			for sj in range(nStates):
				#gamma(t, si) = xi[si,:,t]
				#gamma(sum in t to T-1, si) = xi[si,:,:T-1]
				a[si,sj] = np.sum(xi[si,sj,:T-1])/np.sum(xi[si,:,:T-1])
			for k in range(max(obs)):                    						 #if f:S->O does not preserves dimensions	
				idx = [i for i,x in enumerate(obs) if x==k]
				b[si,k] = np.sum(xi[si,:,idx])/np.sum(xi[si,:,:])
				
		error = np.abs(a-a_old).max()+np.abs(b-b_old).max()
		print(iter, error, np.sum(c), np.sum(alpha[:,T-1]))
	##############################################################
	return pi, a, b	

## Toy examples
Here are the parameters for HMM model.

In [ ]:
transition_data = {'state': ['TV', 'Bar', 'Party', 'Study'],
                          'TV'    : [0.4, 0.3, 0.1, 0.2],
                          'Bar'   : [0.6, 0.05, 0.1, 0.25],
                          'Party' : [0.7, 0.05, 0.05, 0.2],
                          'Study' : [0.3, 0.4, 0.25, 0.05]}

emission_data = {'observation': ['tired', 'hangover', 'anxiety', 'good'],
                        'TV'    : [0.2, 0.1, 0.2, 0.5],
                        'Bar'   : [0.4, 0.2, 0.1, 0.3],
                        'Party' : [0.3, 0.4, 0.2, 0.1],
                        'Study' : [0.3, 0.05, 0.3, 0.35]}

transition_probability = DataFrame(transition_data, columns=['state', 'TV', 'Bar', 'Party', 'Study'])
emission_probability = DataFrame(emission_data, columns=['observation', 'TV', 'Bar', 'Party', 'Study'])

In [ ]:
transition_probability

,state,TV,Bar,Party,Study
0,TV,0.4,0.60,0.70,0.30
1,Bar,0.3,0.05,0.05,0.40
2,Party,0.1,0.10,0.05,0.25
3,Study,0.2,0.25,0.20,0.05


In [ ]:
emission_probability

,observation,TV,Bar,Party,Study
0,tired,0.2,0.4,0.3,0.30
1,hangover,0.1,0.2,0.4,0.05
2,anxiety,0.2,0.1,0.2,0.30
3,good,0.5,0.3,0.1,0.35


In [ ]:
try:
    a = transition_probability.drop(['state'], axis=1).to_numpy()
    b = emission_probability.drop(['observation'], axis=1).to_numpy()
except:
    a = transition_probability.drop(['state'], axis=1).values
    b = emission_probability.drop(['observation'], axis=1).values

In [ ]:
# Assume the initial state probabilites are all equal to 0.25
pi = np.array([0.25, 0.25, 0.25, 0.25])
kibun = np.array(['tired', 'tired', 'good', 'hangover', 'hangover', 'anxiety', 'hangover', 'good'])
obs = np.array([0, 0, 3, 1, 1, 2, 1, 3])

## Problem 4-1. 
What is the probability of seeing current observation?

In [ ]:
####################### YOUR CODE HERE #######################
alpha, c = HMMfwd(pi, a, b, obs)
print(alpha,'\n')
print(alpha[:,-1])
print("Total sequence probability = ", np.sum(alpha[:,-1]))
print('kibun = ',[kibun[i] for i in obs])
##############################################################

[[5.00000000e-02 3.10000000e-02 1.90062500e-02 1.29575938e-03
  2.61938789e-04 6.39928405e-05 5.18474735e-06 5.71393125e-06]
 [1.00000000e-01 2.15000000e-02 5.21625000e-03 1.61205625e-03
  1.31105875e-04 1.13366757e-05 7.01163974e-06 7.94971063e-07]
 [7.50000000e-02 1.12500000e-02 9.84375000e-04 1.50068125e-03
  1.74117422e-04 1.20425619e-05 6.92687805e-06 1.81445352e-07]
 [7.50000000e-02 1.61250000e-02 5.12093750e-03 2.77911719e-04
  4.88098887e-05 3.67284616e-05 9.93883625e-07 1.47872522e-06]] 

[5.71393125e-06 7.94971063e-07 1.81445352e-07 1.47872522e-06]
Total sequence probability =  8.169072883039616e-06
kibun =  ['tired', 'tired', 'hangover', 'tired', 'tired', 'good', 'tired', 'hangover']


## Problem 4-2.
Given the current observation, what is the most likely sequence of states?

In [ ]:
####################### YOUR CODE HERE #######################
print("Maximum probable path = ", Viterbi(pi, a, b, obs)[0])
MAX_PATH = Viterbi(pi, a, b, obs)[0]           #hidden state
STATE = ['TV', 'Bar', 'Party', 'Study']
print("Hidden state = ", [STATE[int(i)] for i in MAX_PATH])
##############################################################

Maximum probable path =  [3. 1. 0. 1. 0. 3. 2. 0.]
Hidden state =  ['Study', 'Bar', 'TV', 'Bar', 'TV', 'Study', 'Party', 'TV']


## Problem 4-3.
Given the current observation and the model, find the model parameters (transition probability, emission probability, initial state probability) that best fits the data.

In [ ]:
####################### YOUR CODE HERE #######################
pi, a, b = BaumWelch(obs,4)
print(pi,'\n',a,'\n',b)
##############################################################

1 1.3457104241992455 8.0 0.7165129626001095
2 0.1878672778807866 8.0 7.539465554828826e-05
3 0.2697638403642004 8.0 6.684423405999526e-05
4 0.24892894338537525 8.0 0.00012359896820549594
5 0.25627277027138107 8.0 0.00017367025272071744
6 0.15950587564992458 8.0 0.0001263826531690465
7 0.1551632236513966 8.0 2.0673984170967046e-05
8 0.2500273035601085 8.0 1.4325493485755378e-06
9 0.2152880600575372 8.0 9.342907477225536e-08
10 0.2106841953004427 8.0 1.3120200970117484e-08
11 0.1994566214199136 8.0 3.469062491965253e-09
12 0.16382415658615945 8.0 1.017518462080823e-09
13 0.12663411476404918 8.0 2.989267271896948e-10
14 0.07259391163752027 8.0 8.051580067756092e-11
15 0.10603766235863715 8.0 1.9118721149033117e-11
16 0.41856922673030367 8.0 4.533950083431576e-12
17 0.027570301358632032 8.0 8.596978578969372e-13
18 0.014307405732266854 8.0 1.7647035280542814e-13
19 0.011001339919015418 8.0 3.724254352181277e-14
20 0.008657817091350717 8.0 7.855577195248331e-15
21 0.006932128965355333 8.0 1

In [ ]:
print(np.sum(a, axis = 1),'\n',np.sum(b, axis = 1))
print(np.sum(b))

[1. 1. 1. 1.] 
 [1.47262131 0.92758882 1.54060006 1.37453172]
5.315341910877056
